# Typesense | Typesense> [Typesense](https://typesense.org)は、オープンソースのインメモリ検索エンジンで、[自己ホスティング](https://typesense.org/docs/guide/install-typesense#option-2-local-machine-self-hosting)することも、[Typesense Cloud](https://cloud.typesense.org/)上で実行することもできます。>> > [Typesense](https://typesense.org) is an open-source, in-memory search engine, that you can either [self-host](https://typesense.org/docs/guide/install-typesense#option-2-local-machine-self-hosting) or run on [Typesense Cloud](https://cloud.typesense.org/).>> Typesenseは、インデックス全体をRAMに格納する（ディスク上にバックアップあり）ことでパフォーマンスに焦点を当てるとともに、利用可能なオプションを簡素化し、良好なデフォルト設定を施すことで、開発者がすぐに使える体験を提供することにも注力しています。>> > Typesense focuses on performance by storing the entire index in RAM (with a backup on disk) and also focuses on providing an out-of-the-box developer experience by simplifying available options and setting good defaults.>> また、属性ベースのフィルタリングとベクトルクエリを組み合わせて、最も関連性の高いドキュメントを取得することができます。>> > It also lets you combine attribute-based filtering together with vector queries, to fetch the most relevant documents.

このノートブックは、TypesenseをVectorStoreとして使用する方法を示しています。> This notebook shows you how to use Typesense as your VectorStore.

まず最初に、依存関係をインストールしましょう：> Let's first install our dependencies:

In [ ]:
!pip install typesense openapi-schema-pydantic openai tiktoken

`OpenAIEmbeddings`を使用するためには、OpenAIのAPIキーを取得する必要があります。> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [6]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Typesense

テストデータセットをインポートしましょう：> Let's import our test dataset:

In [19]:
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "localhost",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "8108",  # Use 443 for Typesense Cloud
        "protocol": "http",  # Use https for Typesense Cloud
        "typesense_api_key": "xyz",
        "typesense_collection_name": "lang-chain",
    },
)

## Similarity Search | 類似性検索

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = docsearch.similarity_search(query)

In [ ]:
print(found_docs[0].page_content)

## Typesense as a Retriever | Typesenseをリトリバーとして使用するTypesenseは、他のベクトルストアと同様に、コサイン類似度を使用するLangChain Retrieverです。> Typesense, as all the other vector stores, is a LangChain Retriever, by using cosine similarity.

In [ ]:
retriever = docsearch.as_retriever()
retriever

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
retriever.get_relevant_documents(query)[0]